!pip install yelp

!pip install simplejson

In [9]:
import io
import simplejson as json
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
# read API keys
with io.open('config_secret.json') as cred:
    creds = json.load(cred)
    auth = Oauth1Authenticator(**creds)
    client = Client(auth)

In [16]:
auth